<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/Double_q_learning_%26_DDQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DDQN

이번 프로젝트는 DDQN 모델을 구현하고 훈련시켜봄으로써 DDQN 아키텍처에 대해 깊이 이해하고 기존 DQN아키텍쳐의 문제점인 행동가치 Q에 대한 과대평가문제가 DDQN아키텍쳐에서 얼마나 완화돼는지 관찰함을 목적으로한다.

# 기존 Q러닝과 이를 활용한 DQN에서 발생하는 행동가치에대한 과대평가 문제.

Q 러닝 알고리즘은 큰 수의 법칙에 따라 Q업데이트를 샘플링에 기반하고, 업데이트에 사용되는 다음상태에서의 최대행동가치의 Q값은 그 다음의 모든 행동을 선택함에 있어서 최적화된 정책을 사용한다고 가정한다.

$ Q_{t+1}(s_t,a_t) = Q_t(s_t,a_t) + \alpha_t(s_t,a_t)(r_t + \gamma max_a Q_t(s_{t+1},a) - Q_t(s_t,a_t)) $

하지만 이런 방식은 큰 문제가 있는데 업데이트에 사용될 다음상태의 최대행동가치가 최적화되지 않은 정책에의해 선택되고 그 가치가 선택된 행동이 따르는 확률분포의 기댓값과는 상당히 다를 수 있다는 것이다.

1.$ E(max_a Q_t(s_{t+1}, a)) $
\
2.$ max_aE(Q_T(s_{t+1}, a)) $

정리하면 우리가 진정으로 필요한것은 2번(각 행동의 행동가치의 기댓값중 최댓값) 이지만 현실적인 한계로 이를 근사하는 1번(최대행동가치의 기댓값)을 사용함으로써 큐값을 추정할때 과대평가가 일어난다는 것 이다.
\
\
\
!과대평가에 대한 직관저인 예시

두개의 주사위를 던진다고 가정할때 각 주사위눈에 대한 기댓값은 3.5, 3.5이므로 기댓값의 최댓값은 3.5일 것 이다. 하지만 두개의 주사위를 던진후 나온 주사위눈중 더 큰값에대한 기댓값은 3.5보다 클 것 이다.

# Double Q Learning과 DDQN이 과대평가를 줄이는 방법.

Double Q Learning에서는 위의 과대평가를 줄이기위해 서로 다른 샘플을 사용해 업데이트되는 두개의 추정기를 사용한다. 두 추정기의 업데이트는 최대값을 갖는 행동을 업데이트되는 추정기가 선택하면 그 행동에 대한 큐값은 다른 추정기가 추정한 값을 사용하여 업데이트된다.

두개의 추정기 $ Q^1 $ 과 $ Q^2 $를 가정했을때 $ Q^1 $의 업데이트는

$ Q^1_{t+1}(s_t,a_t) = Q^1_t(s_t,a_t) + \alpha_t(s_t,a_t)(r_t + \gamma Q^2_t(s_{t+1},argmax_a Q^1_t(s_{t+1}, a)) - Q_t(s_t,a_t)) $

를 따른다.

이를 이용해 DDQN의 엡데이트 수식을 나타내면

$ Y^{DoubleQ}_t = R_{t+1} + \gamma Q(S_{t+1}, argmax_a Q(S_{t+1}, a; \theta_t); \theta '_t) $

으로 표현할 수 있다.
\
\
\
!서로다른 추정기를 사용할때 그 추정치의 최댓값은 기댓값의 최댓값이되며 이는 DDQN논문 Lemma1을 참고.

# 과대평가와 과소평가

위의 Double Q Learning의 방법은 과대평가문제를 해결하지만 반대로 과소평가문제가 발생한다. 하지만 최댓값을 전파하는 Q러닝의 특성상 과대평가된 오차는 계속해서 전파되는 반면 과소평과된 오차는 그런 우려가 적다.

# DDQN아키텍쳐 특징

DDQN의 기본적인 아키텍쳐는 DQN과 동일하다. 여기에 Double Q Learning에서 제시한 방법론에 따라 하나의 추정기가 추가로 필요하지만 이는 기존의 DQN에서 상용되던 Target network를 활용한다.

In [1]:
!pip install gymnasium
!pip install gymnasium[atari]
!pip install gymnasium[accept-rom-license]
!pip install imageio
!pip install imageio-ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446659 sha256=cd2a3cd6bc748971debbc4c7aa17b931d2ad06e8f6cc6a2ddc371fa234c07c5b
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


In [2]:
import gymnasium as gym
import imageio
from gymnasium.wrappers import FrameStack, GrayScaleObservation
import torch
from torch import nn
import numpy as np
import torch.optim as optim
import random
import torch.nn.functional as F
from google.colab import drive
from tqdm.auto import tqdm
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

drive.mount('/content/drive')

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Mounted at /content/drive


util

In [3]:
# 매개변수로 repeat을 추가.(게임 반복 횟수)

def play_game(model, env, repeat, file_name, record : bool = False):

    total_reward = 0
    frames = []

    for i in range(repeat):

        obs, info = env.reset()
        if record == True:
            frames.append(env.render())

        obs, reward, terminated, truncated, info = env.step(1)
        if record == True:
            frames.append(env.render())

        while(terminated == False and truncated == False and info['lives'] == 5):

            _, action = model.greedy_action(torch.tensor(np.array(obs)).to(DEVICE))
            obs, reward, terminated, truncated, info = env.step(action)
            if record == True:
                frames.append(env.render())

            total_reward += reward

    if record == True:
        with imageio.get_writer(f'/content/drive/MyDrive/Colab Notebooks/딥러닝/포트폴리오/DDQN/동영상/{file_name}.mp4', fps=30, ) as video:
            for frame in frames:
                video.append_data(frame)

    total_reward = total_reward / repeat

    return total_reward

# 모델 업데이트.

def model_update(q_model, target_model, optimizer, buffer, batch_size):

    obs_batch, action_batch, reward_batch, nobs_batch = buffer.sample(batch_size)

    obs_batch = torch.tensor(np.array(obs_batch)).float().to(DEVICE)
    action_batch = torch.tensor(np.array(action_batch)).float().to(DEVICE)
    reward_batch = torch.tensor(np.array(reward_batch)).float().to(DEVICE)
    nobs_batch = torch.tensor(np.array(nobs_batch)).float().to(DEVICE)

    with torch.no_grad():
        _, idx = q_model.greedy_action(nobs_batch) # 행동선택은 q모델의 정책에 따른다.
        y = target_model.generate_q(nobs_batch, idx) # 선택된 행동에 대한 q값 평가는 타겟모델로 진행한다.
        y = 0.99*y
        y = y + reward_batch

    q = q_model.generate_q(obs_batch, action_batch)

    loss = F.mse_loss(y, q)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss = loss.item() # .item()은 텐서가 단일값을 포함하고 있을 경우 파이썬의 수자로 변환.

    return loss

def step_and_stack(model, env, buffer, epsilon, step_size):

    obs, reward, terminated, truncated, info = env.step(1)

    for i in range(step_size):

        if terminated == False and truncated == False and info['lives'] == 5:

            action = model.epsilon_greedy_action(torch.tensor(np.array(obs)).to(DEVICE), epsilon)
            nobs, reward, terminated, truncated, info = env.step(action)

            buffer.add([obs, action, reward, nobs])
            obs = nobs

        else:

            obs, info = env.reset()
            nobs, reward, terminated, truncated, info = env.step(1)
            buffer.add([obs, 1, reward, nobs])
            obs = nobs

    return

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Experience replay

In [4]:
class ExperienceReplayMemory:

    def __init__(self, capacity):

        self.capacity = capacity
        self.memory = []

    def add(self, experience):

        if len(self.memory) < self.capacity:
            self.memory.append(experience)

        else:
            self.memory.pop(0) # 리스트가 클 경우 매우 비효율적. 대안으로 deque를 사용할 수 있지만 인데싱이 불편하고 느림.
            self.memory.append(experience)

    def sample(self, batch_size):

        sample = random.sample(self.memory, batch_size)

        obs_batch = [obs for obs, _, _, _ in sample]
        action_batch = [action for _, action, _, _ in sample]
        reward_batch = [reward for _, _, reward, _ in sample]
        nobs_batch = [nobs for _, _, _, nobs in sample]

        return obs_batch, action_batch, reward_batch, nobs_batch

    def __len__(self):

        return len(self.memory)

DDQN아키텍쳐

Note:

1. 모델 구조

레이어구조는 기존 2013 DQN논문의 것에서 아키텍쳐와 하이퍼파라미터에 대해 더 자세히 설명되어있는 2015년 논문의 것으로 수정했다.

In [5]:
class DDQN(nn.Module):
    def __init__(self):

        super().__init__()

        self.conv = nn.Sequential(nn.Conv2d(4, 32, kernel_size = 8, stride = 4),
                                  nn.ReLU(),
                                  nn.Conv2d(32, 64, kernel_size = 4, stride = 2),
                                  nn.ReLU(),
                                  nn.Conv2d(64, 64, kernel_size = 3, stride = 1),
                                  nn.ReLU())

        self.fc = nn.Sequential(nn.Linear(3136, 512),
                                nn.ReLU(),
                                nn.Linear(512, 4))

    def forward(self, x):

        with torch.no_grad():
            x = self.preprocessing(x)

        x = self.conv(x)
        x = torch.flatten(x, start_dim = 1)
        x = self.fc(x)

        return x

    def greedy_action(self, x):

        with torch.no_grad():
            x = self.forward(x)
            x, idx = torch.max(x, 1)

        return x, idx

    def epsilon_greedy_action(self, x ,epsilon):

        with torch.no_grad():
            if random.random() > epsilon:
                x, idx = self.greedy_action(x)
                x = idx.item()

            else:
                x = random.randrange(4)

        return x

    def generate_q(self, x, action_batch):

        x = self.forward(x)
        action_batch = action_batch.unsqueeze(1).long()
        q = torch.gather(x, 1, action_batch)
        q = q.squeeze(1)

        return q

    def preprocessing(self, x):

        # x = np.array(x)
        # x = torch.tensor(x)

        if x.ndim == 3:
            x = x.unsqueeze(0)

        x = x[:, :, 34:-16, :]
        resize_transform = transforms.Resize((84, 84))
        x = resize_transform(x)
        x = x.float() / 255.

        return x

Q모델과 타겟모델 생성.

In [6]:
q_model = DDQN().to(DEVICE)
t_model = DDQN().to(DEVICE)
t_model.load_state_dict(q_model.state_dict()) # Q모델의 파라미터를 타겟모델에 복사한다.

<All keys matched successfully>

환경 초기화.

In [7]:
#환경 초기화
env = gym.make("ALE/Breakout-v5", render_mode='rgb_array', obs_type='grayscale')
env = FrameStack(env, 4)
obs, info = env.reset()

하이퍼파라미터 초기화.

In [8]:
# 매개변수 초기화.
epsilon = 1
buffer = ExperienceReplayMemory(1000000)
optimizer = optim.RMSprop(q_model.parameters(), lr=0.00025, alpha=0.95, eps=1e-6)
reward_r = []
loss_r = []
epsilon_r = []

훈련 루프

Note

2. Q모델과 타겟모델의 업데이터 주기.

Q모델의 업데이트 주기는 10스텝, 타겟모델의 업데이트 주기는 5000스텝으로 설정했다. 2015 DQN논

3. 훈련 결과.

훈련루프는 모델업데이트 기준으로 총 100만step진행했으며 20시간이 소요 됐다. 에이전트의 성능은 변도성이 크지만 꾸준히 상향됬다. 하지만 아쉽게도 성능이 수렴하는 구간까지 훈련을 진행하지는 못 했다.

In [9]:
step_and_stack(q_model, env, buffer, epsilon, step_size = 50000)

for i in tqdm(range(1000000)):

    step_and_stack(q_model, env, buffer, epsilon, step_size = 10)

    loss = model_update(q_model, t_model, optimizer, buffer, 64)
    loss_r.append(loss)

    if i % 5000 == 0:

        t_model.load_state_dict(q_model.state_dict())
        torch.save(q_model, f'/content/drive/MyDrive/Colab Notebooks/딥러닝/포트폴리오/DDQN/model/{str(i)}.pth')

        r = play_game(q_model, env, 5,  str(i), record = True)

        print(f"total reward : {r}")
        reward_r.append(r)

        # 훈련 과정 백업.

        # reward_r 리스트를 reward.txt 파일에 저장
        with open('/content/drive/MyDrive/Colab Notebooks/딥러닝/포트폴리오/DDQN/log/reward.txt', 'w', encoding='utf-8') as file:
            for item in reward_r:
                file.write("%s\n" % item)

        # loss_r 리스트를 loss.txt 파일에 저장
        with open('/content/drive/MyDrive/Colab Notebooks/딥러닝/포트폴리오/DDQN/log/loss.txt', 'w', encoding='utf-8') as file:
            for item in loss_r:
                file.write("%s\n" % item)

        # epsilon_r 리스트를 epsilon.txt 파일에 저장
        with open('/content/drive/MyDrive/Colab Notebooks/딥러닝/포트폴리오/DDQN/log/epsilon.txt', 'w', encoding='utf-8') as file:
            for item in epsilon_r:
                file.write("%s\n" % item)

    if i % 500 == 0:
        epsilon = max(epsilon - 0.00045, 0.1)
        epsilon_r.append(epsilon)

  0%|          | 0/1000000 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


total reward : 0.8


total reward : 0.6


total reward : 0.4


...

total reward : 5.4


total reward : 12.0


total reward : 9.6


total reward : 4.6


total reward : 6.2


In [10]:
# # reward_r 리스트로 데이터를 불러오기
# reward_r = []
# with open('/content/drive/MyDrive/Colab Notebooks/딥러닝/포트폴리오/DDQN/log/reward.txt', 'r', encoding='utf-8') as file:
#     for line in file:
#         reward_r.append(float(line.strip()))

# # loss_r 리스트로 데이터를 불러오기
# loss_r = []
# with open('/content/drive/MyDrive/Colab Notebooks/딥러닝/포트폴리오/DDQN/log/loss.txt', 'r', encoding='utf-8') as file:
#     for line in file:
#         loss_r.append(float(line.strip()))

# # epsilon_r 리스트로 데이터를 불러오기
# epsilon_r = []
# with open('/content/drive/MyDrive/Colab Notebooks/딥러닝/포트폴리오/DDQN/log/epsilon.txt', 'r', encoding='utf-8') as file:
#     for line in file:
#         epsilon_r.append(float(line.strip()))

In [11]:
# plt.plot(reward_r, label = 'Reward', color = 'red')
# plt.xlabel('Step(5000)')
# plt.ylabel('Reward')
# plt.title('10p Reward')
# plt.legend()

# plt.show()


강화학습에서 하이퍼파라미터 탐색에 사용될 수 있는 여러 방법들이 있으며, 이들은 계산 비용, 탐색 공간의 크기, 그리고 필요한 정밀도에 따라 선택됩니다. 그리드 서치(Grid Search)가 전통적이고 간단한 방법이지만 높은 계산 비용 때문에 실제 큰 문제에는 적합하지 않을 수 있습니다. 따라서 다음과 같은 대안적인 방법들이 고려될 수 있습니다:

랜덤 서치(Random Search): 랜덤 서치는 하이퍼파라미터의 값들을 무작위로 선택하여 탐색 공간 내에서 여러 구성을 시도합니다. 이 방법은 간단하며 때로는 예상치 못한 좋은 결과를 빠르게 찾아낼 수 있습니다. 계산 비용이 높은 그리드 서치에 비해 효율적일 수 있습니다.

베이지안 최적화(Bayesian Optimization): 이 방법은 성능 함수의 과거 평가를 기반으로 하여 하이퍼파라미터의 가장 유망한 값들을 예측합니다. 이 방법은 비교적 적은 수의 평가로 좋은 성능을 찾을 수 있도록 설계되었으며, 복잡한 하이퍼파라미터 공간에서 효과적일 수 있습니다.

진화 알고리즘(Evolutionary Algorithms): 이들은 자연 선택과 유전학의 원리를 모방하여 최적의 하이퍼파라미터 구성을 "진화"시키는 방법입니다. 초기에 임의로 선택된 하이퍼파라미터 세트는 성능에 따라 선택되고 교차 및 변이를 통해 새로운 세트를 생성합니다. 이 과정은 특정 조건이 충족될 때까지 반복됩니다.

코사인 어닐링(Cosine Annealing): 이 방법은 초기에 높은 탐색 범위에서 시작하여 시간이 지남에 따라 점차 탐색 범위를 줄여나가는 방식으로 작동합니다. 이는 물리학에서 어닐링 과정을 모방한 것으로, 최적화 과정에서 전역 최소값에 접근할 수 있게 합니다.

하이퍼밴드(Hyperband): 하이퍼밴드는 리소스 할당의 효율성을 최대화하기 위해 설계된 방법입니다. 다양한 구성에 대한 초기 평가를 수행한 후, 가장 잘 수행되는 구성에 더 많은 리소스를 집중적으로 할당합니다.

강화학습 프로젝트에서는 이러한 방법들 중 하나 또는 여러 개를 조합하여 하이퍼파라미터 탐색을 수행할 수 있습니다. 선택된 방법은 특정 문제의 특성, 사용 가능한 계산 자원, 그리고 원하는 실험의 속도와 정확도에 따라 달라질 수 있습니다.